In [ ]:
# Loading required libraries and functions
import os
import shutil
import random
import itertools
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib as mpl
from keras import backend
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from keras.applications import imagenet_utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, Activation
from sklearn.metrics import precision_score, recall_score
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import decode_predictions, preprocess_input

In [ ]:
!unzip '/content/data (1).zip' -d '/content'


In [ ]:
# Loading data and preprocessing images according to mobilenet requirements
# Creating batches of data

labels = ['Flooding', 'No Flooding']
train_path = '/content/data/train'
valid_path = '/content/data/valid'
test_path = '/content/data/test'

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224,224), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=10, shuffle=False)

In [ ]:
#Loading pre-trained lightweight mobilenet image classifier
mobile = tf.keras.applications.mobilenet.MobileNet()


In [ ]:
mobile.summary()

In [ ]:
# Store all layers of the original mobilenet except the last 5 layers in variable x
# There is no predefined logic behind this, it just gives the optimal results for this task
# Also, we will be only training the last 5 layers of the mobilenet during finetuning as we want 
# it to keep all of the previously learned weights 
x = mobile.layers[-6].output

In [ ]:

# Create an output layer with binary output layer, as we want our model to be a binary classifier, 
# i.e. to classify flooding and no flooding
output = Dense(units=2, activation='softmax')(x)

In [ ]:

# Construct the new fine-tuned mode
model = Model(inputs=mobile.input, outputs=output)

In [ ]:

# Freez weights of all the layers except for the last five layers in our new model, 
# meaning that only the last five layers of the model will be trained.
for layer in model.layers[:-23]:
    layer.trainable = False

In [ ]:
#model.summary()


In [ ]:

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the Model

model.fit(x=train_batches,
          steps_per_epoch=len(train_batches),
          validation_data=valid_batches,
          validation_steps=len(valid_batches),
          epochs=10,
          verbose=2
)



In [ ]:
# Saving and loading our trained for future use

model.save("fine_tuned_flood_detection_model.model")
# model.load_weights('fine_tuned_flood_detection_model')

In [ ]:
# Make predictions and plot confusion matrix to look how well our model performed in classifying 
# flooding and no flooding images 

test_labels = test_batches.classes
predictions = model.predict(x=test_batches, steps=len(test_batches), verbose=0)
cm = confusion_matrix(y_true=test_labels, y_pred=predictions.argmax(axis=1))
precision = precision_score(y_true=test_labels, y_pred=predictions.argmax(axis=1))
f1_score = f1_score(y_true=test_labels, y_pred=predictions.argmax(axis=1))
accuracy = accuracy_score(y_true=test_labels, y_pred=predictions.argmax(axis=1))
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Pring precision, F1 score and accuracy of our model
print('Precision: ', precision)
print('F1 Score: ', f1_score)
print('Accuracy: ', accuracy)

In [ ]:
# Confusion Matrix 
test_batches.class_indices
cm_plot_labels = ['Flooding','No Flooding']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

In [ ]:
# Prepare image for mobilenet prediction

def preprocess_image(file):
    img_path = '/content/'
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [ ]:
# Display image which we want to predict
from IPython.display import Image
Image(filename='2.jpg', width=300,height=200)

In [ ]:
# Preprocess image and make prediction

preprocessed_image = preprocess_image('3.jpeg')
predictions = model.predict(preprocessed_image)
# Print predicted accuracy scores for both classes, i.e. (1) Flooding, (2) No Flooding
#predictions
# Get the maximum probability score for predicted class from predictions array
result = np.argmax(predictions)
# Print the predicted class label
labels[result]

In [ ]:
import cv2
CATEGORIES = ['Flooding', 'No Flooding']

model = tf.keras.models.load_model("/content/fine_tuned_flood_detection_model.model")
#prediction = model.predict('/content/3.jpeg')


In [ ]:
preprocessed_image = preprocess_image('1.jpg')
prediction = model.predict(preprocessed_image)
print(CATEGORIES[int(prediction[0][0])])
#result = np.argmax(prediction)
# Print the predicted class label
#labels[result]

In [ ]:
# import the necessary packages
from tensorflow.keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle
import cv2

In [ ]:
# load the trained model and label binarizer from disk
print("[INFO] loading model and label binarizer...")
model = load_model("/content/fine_tuned_flood_detection_model.model")
#lb = pickle.loads(open(["label_bin"], "rb").read())
# initialize the image mean for mean subtraction along with the
# predictions queue
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
#Q = deque(maxlen["size"])

In [ ]:
vs = cv2.VideoCapture('/content/videoplayback.mp4')
writer = None
(W, H) = (None, None)
# loop over frames from the video file stream
while True:
	# read the next frame from the file
	(grabbed, frame) = vs.read()
	# if the frame was not grabbed, then we have reached the end
	# of the stream
	if not grabbed:
		break
	# if the frame dimensions are empty, grab them
	if W is None or H is None:
		(H, W) = frame.shape[:2]

In [ ]:
output = frame
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
frame = cv2.resize(frame, (224, 224)).astype("float32")
frame -= mean